In [1]:
import numpy as np
import keras
from keras import layers
import string
import random
import sys
from utils import log, generate_overlapping_sequences, load_short_jokes_corpus

Using TensorFlow backend.


In [2]:
jokes, text = load_short_jokes_corpus(limit=None)
print 'Corpus Length: {c}'.format(c=len(text))


Corpus Length: 21787187


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(jokes)
vocab_size = len(tokenizer.word_index) + 1

print('Vocab Size', vocab_size)

('Vocab Size', 70649)


In [4]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds.astype('float64'))
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [5]:
def generate_overlapping_encoded_sequences(jokes, maxlen, step):
    sentences = []
    next_words = [] # holds the targets
    for joke in jokes:
        for i in range(0, len(joke) - maxlen, step):
            sentences.append(joke[i: i + maxlen])
            next_words.append(joke[i + maxlen])
    return sentences, next_words

In [6]:
# split jokes into sequences of 5

integer_encoded_docs = tokenizer.texts_to_sequences(jokes)
split_encoded_docs, next_words = generate_overlapping_encoded_sequences(integer_encoded_docs, 15, 2)
padded_docs = pad_sequences(split_encoded_docs, padding='pre')
# need to get next word for each of these 
next_words = np.asarray(next_words)
print("Number of Sequences:", len(padded_docs))

('Number of Sequences:', 515523)


In [7]:
print 'Vectorization.'
y = np.zeros((len(padded_docs), vocab_size), dtype=np.bool)
for i, padded_doc in enumerate(padded_docs):
    y[i, next_words[i]] = 1

Vectorization.


In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.optimizers import RMSprop
from keras.layers.embeddings import Embedding

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=padded_docs.shape[1]))
model.add(Bidirectional(LSTM(128, dropout=0.01, recurrent_dropout=0.1, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.01, recurrent_dropout=0.1, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.01, recurrent_dropout=0.1, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(2048, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 100)           7064900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 15, 256)           234496    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 15, 256)           394240    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 15, 256)           394240    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              526336    
_________________________________________________________________
dense_2 (Dense)              (None, 70649)             144759801 
Total para

In [10]:
word_index = tokenizer.word_index
index_to_word = dict((index, word) for word, index in word_index.items())
max_words = 5
maxlen = padded_docs.shape[1]

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds.astype('float64'))
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
for i in range(1, 20):
    print 'Epoch: {i}'.format(i=i)
    model.fit(padded_docs, y, batch_size=512, nb_epoch=1) 
    if i >= 3:
        model.save_weights('jokes_weights_512_with_reg_nodes_{i}_iter'.format(i=i), overwrite=True)
    print '\n'
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print '\n------------ temperature: {t}'.format(t=temperature)
        start_index = random.randint(0, len(text.split(' ')) - max_words - 1)
        generated_text = " ".join(text.split(' ')[start_index: start_index + max_words])
        integer_encoded_gen_text = tokenizer.texts_to_sequences([generated_text])
        readable_gen_text = " ".join(map(lambda key: index_to_word[key], integer_encoded_gen_text[0]))
        print  '-----  Generating with Seed: "{gen}"'.format(gen=readable_gen_text)
        print '\n'
        sys.stdout.write(readable_gen_text)

        for i in range(35):
            integer_encoded_gen_text = tokenizer.texts_to_sequences([generated_text])
#             print('Integer Encoded', integer_encoded_gen_text)
            readable_gen_text = " ".join(map(lambda key: index_to_word[key], integer_encoded_gen_text[0]))
            padded_gen_text = pad_sequences(integer_encoded_gen_text, maxlen=maxlen, padding='pre')
            preds = model.predict(padded_gen_text, verbose=0)[0]

            next_index = sample(preds, temperature)
            most_probable_next_word = index_to_word[next_index]
#             print('Generated:', generated_text, 'Next: ', most_probable_next_word)
            generated_text += " " + most_probable_next_word
            generated_text = " ".join(generated_text.split(' ')[1:])
            sys.stdout.write(" " + most_probable_next_word)
    

Epoch: 1
Epoch 1/1
515523/515523 [==============================] - 359s - loss: 7.8211   



------------ temperature: 0.2
-----  Generating with Seed: "ok keith we had to"


ok keith we had to a a a the the the the the the the the the the a the the a the the a the the a the a a the the the the the a a the a
------------ temperature: 0.5
-----  Generating with Seed: "how do you know a"


how do you know a a believe the the and i the a a no the a a i'm man a a the a the the just the is let there me got a a that to a fish my
------------ temperature: 1.0
-----  Generating with Seed: "window at the driveway and"


window at the driveway and blonde human man donkey help fall funny girlfriends first it nothing questions forgot bartender mouth just well drive drink came hope two ask they're eye serious five started whispered jokes large donuts you store hydrate
------------ temperature: 1.2
-----  Generating with Seed: "smoking some weed hibearnation i"


smoking some weed hibearnation i by

r relationships you'll never have a the other of the other of a the other of the other of the other and i was the the other and a the other of the other of the other of the other
------------ temperature: 0.5
-----  Generating with Seed: "giant bird goes to a"


giant bird goes to a a guy and i have a to into a other of the way i was an kid of the way who i told me i said a man was it and going to be my
------------ temperature: 1.0
-----  Generating with Seed: "the difference between a dirty"


the difference between a dirty head it get in one year bulb you got and bunch

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 to know he was my mother do the wife is the up and probably the body on your right people over it bad from
------------ temperature: 1.2
-----  Generating with Seed: "her me we were on"


her me we were on 5 money against some work tough beach my hour why everything legs tuesday doesn't think was going to which you're and how you then no re true the rest of a day and probably turnEpoch: 9
Epoch 1/1
515523/515523 [==============================] - 357s - loss: 6.9520   



------------ temperature: 0.2
-----  Generating with Seed: "with me saying there are"


with me saying there are a other and they have a the other of the other of it and a the other of the other of a name and i have a the other of a the other of the
------------ temperature: 0.5
-----  Generating with Seed: "can't stevie wonder see his"


can't stevie wonder see his car and the wife i don't know i didn't it died any other to say i have a same name of the house and just be as do see a other you was a it
------------ t

the keyboard out is it room be trying to go up it down come he called my fridge another punch little 6 now having the and mine i'd say there are happy either room they almost waits sorry no listEpoch: 16
Epoch 1/1
515523/515523 [==============================] - 357s - loss: 6.9525   



------------ temperature: 0.2
-----  Generating with Seed: "as girl stingray you know"


as girl stingray you know to be the the the a the other of the of of the other and a and i be a the the the year of the family and i have a and i have a
------------ temperature: 0.5
-----  Generating with Seed: "posts on reddit are r jokes"


posts on reddit are r jokes no to be the wife in the second in the head in the car of a sentence i don't know i have a to get and he couldn't be the head to the room i
------------ temperature: 1.0
-----  Generating with Seed: "deal with all these cyberpersons"


deal with all these cyberpersons then if he like saying it replies as dead empty at the car pet more easy baby but

In [14]:
# Testing
model.load_weights('jokes_weights_512_with_reg_nodes_19_iter')
for i in range(20, 60):
    print 'Epoch: {i}'.format(i=i)
    model.fit(padded_docs, y, batch_size=512, nb_epoch=1) 
    if i >= 3:
        model.save_weights('jokes_weights_512_with_reg_nodes_{i}_iter'.format(i=i), overwrite=True)
    print '\n'
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print '\n------------ temperature: {t}'.format(t=temperature)
        start_index = random.randint(0, len(text.split(' ')) - max_words - 1)
        generated_text = " ".join(text.split(' ')[start_index: start_index + max_words])
        integer_encoded_gen_text = tokenizer.texts_to_sequences([generated_text])
        readable_gen_text = " ".join(map(lambda key: index_to_word[key], integer_encoded_gen_text[0]))
        print  '-----  Generating with Seed: "{gen}"'.format(gen=readable_gen_text)
        print '\n'
        sys.stdout.write(readable_gen_text)

        for i in range(35):
            integer_encoded_gen_text = tokenizer.texts_to_sequences([generated_text])
#             print('Integer Encoded', integer_encoded_gen_text)
            readable_gen_text = " ".join(map(lambda key: index_to_word[key], integer_encoded_gen_text[0]))
            padded_gen_text = pad_sequences(integer_encoded_gen_text, maxlen=maxlen, padding='pre')
            preds = model.predict(padded_gen_text, verbose=0)[0]

            next_index = sample(preds, temperature)
            most_probable_next_word = index_to_word[next_index]
#             print('Generated:', generated_text, 'Next: ', most_probable_next_word)
            generated_text += " " + most_probable_next_word
            generated_text = " ".join(generated_text.split(' ')[1:])
            sys.stdout.write(" " + most_probable_next_word)

            

Epoch: 20
Epoch 1/1
515523/515523 [==============================] - 357s - loss: 6.9648   



------------ temperature: 0.2
-----  Generating with Seed: "as this joke i bought"


as this joke i bought a time i have a the the other the of the the ass in the other of the the face and i don't know i just i was a the face is a the the
------------ temperature: 0.5
-----  Generating with Seed: "daddy bullet a we're gonna"


daddy bullet a we're gonna then have a the car of a own as his airport does to you hit it for the time i'm a work the life and he was a of time with that one on the
------------ temperature: 1.0
-----  Generating with Seed: "why did the why did"


why did the why did turn the ball them so i wouldn't listen and shut up if no line that his state right cream just disappointed night cell owner sit with that 000 five again now on being forever okay
------------ temperature: 1.2
-----  Generating with Seed: "is selfish he does a"


is selfish he does a d law that's going to o

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until





my own eyes snow white for to eagle i wants down hard whatever on walk of bed and enough the end for and not 2 so she's prove hey throwing are a paper lets you be how says man yeahEpoch: 22
Epoch 1/1
515523/515523 [==============================] - 526s - loss: 6.9632   



------------ temperature: 0.2
-----  Generating with Seed: "doubt i'd remind him he"


doubt i'd remind him he was a the the place to be a the shit to be a and i have a the other the the wife is a the man was a the car i don't know to be
------------ temperature: 0.5
-----  Generating with Seed: "race between the horses with"


race between the horses with the car me in the other of the house about the the other the face says to a the top the father who does the time no a one of my other day to a
------------ temperature: 1.0
-----  Generating with Seed: "new studies revealed 9"


new studies revealed 9 i was kids i ever thought you can be her and didn't call the it taking this afternoon often right work in now r

already accepted the fact that says will have a with best rock before son special and staring that spelling they'll it he said i blew work years two what i say is they week it was you getting bread 911Epoch: 29
Epoch 1/1
515523/515523 [==============================] - 358s - loss: 6.9987   



------------ temperature: 0.2
-----  Generating with Seed: "stick together we can stop"


stick together we can stop i have a the head i can be a and i don't be a the the of the the other and a the other a the head i said i have a the face i
------------ temperature: 0.5
-----  Generating with Seed: "do you have mcdonald's money"


do you have mcdonald's money and a pool it is a the shit a outside the way i really i got a first one in a child i don't have the station the other bear i give the couch but
------------ temperature: 1.0
-----  Generating with Seed: "all i remember is that"


all i remember is that i'm just why what she's under the buck patient do am at a bunch of all her food from th

interview what's your greatest weakness be she is a record a dick sex with big petting light side for pretty down an face up on yard if are 4 edit saying to keep 5 oh boy poop of to changeEpoch: 36
Epoch 1/1
515523/515523 [==============================] - 449s - loss: 7.0179   



------------ temperature: 0.2
-----  Generating with Seed: "really ask him how"


really ask him how i am a the other of the other of the other of the other the other is a and i know to have a the other of a time and i don't know i know
------------ temperature: 0.5
-----  Generating with Seed: "why were the teachers eyes"


why were the teachers eyes i was a of it but on a thing to get to a year one is a friend but to be a it was a house the other is not a car and a the
------------ temperature: 1.0
-----  Generating with Seed: "obsession with pointing out doors"


obsession with pointing out doors hell in your light other idea to hear and to have going to have over right they're a really man does you think

whats great about alzhemiers nevermind they did saw your head wouldn't just doesn't be just the blood taking and won't call one like any inches blowing too shot five load in my trees to ask you came i feel intoEpoch: 43
Epoch 1/1
515523/515523 [==============================] - 806s - loss: 7.0504   



------------ temperature: 0.2
-----  Generating with Seed: "dig that kind of thing"


dig that kind of thing i have a the the other the other the other i have a the time in the other of the the the room to be a the face i have the the face to be
------------ temperature: 0.5
-----  Generating with Seed: "fuuuuck yeah shakes pill bottle"


fuuuuck yeah shakes pill bottle of one of the phone and going to do the mouth has you had a shit to be i can get me to be the hole and to see it now you got my room
------------ temperature: 1.0
-----  Generating with Seed: "underwear is under there dad"


underwear is under there dad meant there he can think i got on least walk back at her spider a sa

how many women have you norris a their body the clown for again stand of the people they be crying it beef seeing sound by math answer because did she give them again with the shit has tried bill saysEpoch: 50
Epoch 1/1
515523/515523 [==============================] - 907s - loss: 7.0240   



------------ temperature: 0.2
-----  Generating with Seed: "coffee oh no i really"


coffee oh no i really have a the other the face was a the face to the other the house was a of the other the life to be a and i said i am a the wife was a
------------ temperature: 0.5
-----  Generating with Seed: "mormon cats have 9 wives"


mormon cats have 9 wives for the family he was just in the same the a table get a the life to be a it is the phone so he was a a right front the joke of the way
------------ temperature: 1.0
-----  Generating with Seed: "to believe that all this"


to believe that all this side as about you old national girl don't be turned up in the car inches hard to see their up 6 why how

months weeks years doctor nine in the toaster me no 3 fish that her paper long stupid child punch girls called to looks inside and ok to had yeah they need was a sausage kick me a nuts of himEpoch: 57
Epoch 1/1
515523/515523 [==============================] - 881s - loss: 7.0753   



------------ temperature: 0.2
-----  Generating with Seed: "what it takes to do"


what it takes to do a in the other a the other the of the wife and a ass i don't know i don't know i think i said i don't know i was a of the other the of
------------ temperature: 0.5
-----  Generating with Seed: "on fire and he'll be"


on fire and he'll be you're just a on his face it's a first way i was a time to be a out of the other the bit the car and ask a job and a it to be this
------------ temperature: 1.0
-----  Generating with Seed: "told my boyfriend that a"


told my boyfriend that a before coming into him do me yes you should the of fire move please decided it watch both pulls time see i said has nah eye twi

In [15]:
model.load_weights('jokes_weights_512_with_reg_nodes_59_iter')
for i in range(60, 120):
    print 'Epoch: {i}'.format(i=i)
    model.fit(padded_docs, y, batch_size=512, nb_epoch=1) 
    if i >= 3:
        model.save_weights('jokes_weights_512_with_reg_nodes_{i}_iter'.format(i=i), overwrite=True)
    print '\n'
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print '\n------------ temperature: {t}'.format(t=temperature)
        start_index = random.randint(0, len(text.split(' ')) - max_words - 1)
        generated_text = " ".join(text.split(' ')[start_index: start_index + max_words])
        integer_encoded_gen_text = tokenizer.texts_to_sequences([generated_text])
        readable_gen_text = " ".join(map(lambda key: index_to_word[key], integer_encoded_gen_text[0]))
        print  '-----  Generating with Seed: "{gen}"'.format(gen=readable_gen_text)
        print '\n'
        sys.stdout.write(readable_gen_text)

        for i in range(35):
            integer_encoded_gen_text = tokenizer.texts_to_sequences([generated_text])
#             print('Integer Encoded', integer_encoded_gen_text)
            readable_gen_text = " ".join(map(lambda key: index_to_word[key], integer_encoded_gen_text[0]))
            padded_gen_text = pad_sequences(integer_encoded_gen_text, maxlen=maxlen, padding='pre')
            preds = model.predict(padded_gen_text, verbose=0)[0]

            next_index = sample(preds, temperature)
            most_probable_next_word = index_to_word[next_index]
#             print('Generated:', generated_text, 'Next: ', most_probable_next_word)
            generated_text += " " + most_probable_next_word
            generated_text = " ".join(generated_text.split(' ')[1:])
            sys.stdout.write(" " + most_probable_next_word)


Epoch: 60
Epoch 1/1
515523/515523 [==============================] - 547s - loss: 7.1088   



------------ temperature: 0.2
-----  Generating with Seed: "love of adele goodbye it's"


love of adele goodbye it's a of the car

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 i was a the other and and i just is a day i are a day to have a first of one the other i have a of a of it
------------ temperature: 0.5
-----  Generating with Seed: "she got the strawberry fields"


she got the strawberry fields a ass to they get to i even get a night and in her day in the face to do of the guy says i am he you was i don't call it out of
------------ temperature: 1.0
-----  Generating with Seed: "world's slowest vampire i got"


world's slowest vampire i got has women on a other baby before you never don't pick sex were i live the little over to her kid as their joke how while for a field do you know that i been
------------ temperature: 1.2
-----  Generating with Seed: "says doctor doctor my penis"


says doctor doctor my penis a part old year i lost yet in parent and and let me drive enough you what's a story he'll see mom were i were right really once it thought it will fit fuck apparentlyEpoch: 61
Epoch 1/1
515523/515523 [==============================] - 782s - 

gay because he blueberry if you have a year of the car to be a dog i have a little time and a second wife a time and in the mouth and the wife was the first time to be
------------ temperature: 0.5
-----  Generating with Seed: "it only had one"


it only had one i just looked to be too the other is the other is the other of the giant work and the because it was and but i want to be you i'll be a other time
------------ temperature: 1.0
-----  Generating with Seed: "know he was suicidal when"


know he was suicidal when i don't know i probably got the four doctor who do i just come to it what do better are its he's that's and die to make a more my train too at the car
------------ temperature: 1.2
-----  Generating with Seed: "him a bad turn what"


him a bad turn what i started the mouth will kill lol haha i'm a blood you know don't check it to my my coffee no he think i can want that pretty me to figure off the side iEpoch: 68
Epoch 1/1
515523/515523 [==============================] -

runs in your jeans what was the time to the man to the time to the ass to the house i was a man i was a time i don't know to the job i was the time to the
------------ temperature: 0.5
-----  Generating with Seed: "well i am seeing less"


well i am seeing less life of still a c is sweet the years and the time in it i get the of the room and i'm has you would a the time was the same guy in the life
------------ temperature: 1.0
-----  Generating with Seed: "while white people clap"


while white people clap not 5 week don't tell to step it from how around and a joke i don't don't hold around if for me i can't was potato a punchline not so very guys been 40 so
------------ temperature: 1.2
-----  Generating with Seed: "hearts during the national anthem"


hearts during the national anthem out down i'm no an one only of this would enjoy who yelling is pack goes actually much already one this she come the face with and 12 when i like the boss so that'sEpoch: 75
Epoch 1/1
515523/515523 [==

...

is a parent like a blonde and a other thing this is the kids and the day was a car to get a life a bar in a kitchen a job wife and and from the same time i don't
------------ temperature: 1.0
-----  Generating with Seed: "the similarity between an old"


the similarity between an old woman sex in the mirror is all two i'm you dude getting full of it takes 2 is tripping in the tzu my bathroom doesn't start of me i'll run them he was her where
------------ temperature: 1.2
-----  Generating with Seed: "thinner lately it's lost several"


thinner lately it's lost several ya crack out of i just have he's me cold oh cop in it how i'll know who asked at me and facebook their diet doesn't her like 5 no school out i've figured anEpoch: 89
Epoch 1/1
515523/515523 [==============================] - 445s - loss: 7.2054   



------------ temperature: 0.2
-----  Generating with Seed: "the scarecrow win his award"


the scarecrow win his award for the other one was a time and a joke and i was a lit

-----  Generating with Seed: "play on people in their"


play on people in their way from it with my white today i can them all don't actually the new dark idiot were all you use from being had ate common goes because did you sure about a to find
------------ temperature: 1.2
-----  Generating with Seed: "call a frog in the"


call a frog in the top first time for its way takes is the rest of a lottery kid so that's never put this time and it doesn't kill me got he is a total old major it more inEpoch: 96
Epoch 1/1
515523/515523 [==============================] - 579s - loss: 7.2283   



------------ temperature: 0.2
-----  Generating with Seed: "lips said no but her"


lips said no but her a other i was a other time the other of the other of the other of the other is a joke and the other of the other of the other and a man and the
------------ temperature: 0.5
-----  Generating with Seed: "the title what's the worst"


the title what's the worst house like the life are a bit and to h



use swear words then apologise to have at me the eyes much living to avoid ever horse hole had with his boat when if you there's is called my roof many i play for twitter i'll do of my load
------------ temperature: 1.2
-----  Generating with Seed: "don't think you have enough"


don't think you have enough gift d i calls around that often have have becomes guilty who hate chuck jesus all that in them less color decided of your garage jump to go us it a year old shit schoolEpoch: 103
Epoch 1/1
515523/515523 [==============================] - 492s - loss: 7.2875   



------------ temperature: 0.2
-----  Generating with Seed: "during a 69 that's how"


during a 69 that's how i got a other the other and a time the other is a other time the other and a other is a other is the other of the other of the other of the other
------------ temperature: 0.5
-----  Generating with Seed: "walk into a plane ouch"


walk into a plane ouch to be a man with the dog and my car for it of the other joke

i feel a spree coming to do an hands the nose we're she gun that for a other it all said he's actually ten saying you asks sorry can worry i almost never me of it to i you've won't
------------ temperature: 1.2
-----  Generating with Seed: "better i was going to"


better i was going to buy sit about only mind with them asked can't already some of half as i this place on holding 1 easier too flying that is if is finally ten said out but you get theEpoch: 110
Epoch 1/1
515523/515523 [==============================] - 595s - loss: 7.3605   



------------ temperature: 0.2
-----  Generating with Seed: "why do seagulls fly over"


why do seagulls fly over to a other is a other is to the other one i don't be the other is a other is a other of the other of a wife to the other time i don't have
------------ temperature: 0.5
-----  Generating with Seed: "weeks ago so far i've"


weeks ago so far i've shit his doctor a new dick for the dog at the mouth of to the other is the other one are a ma

he didn't truss it knock watching some it is you're but like a game of things back when you have it was of a bitch has my word with my job all on a work before she must leave and
------------ temperature: 1.2
-----  Generating with Seed: "way i can sell young"


way i can sell young he ask sleeping they do an salt hit much later says sort in a ice world out was made up nobody in common far and shake says there away make and pretty you prefer exactlyEpoch: 117
Epoch 1/1
515523/515523 [==============================] - 541s - loss: 7.3772   



------------ temperature: 0.2
-----  Generating with Seed: "see a black cat"


see a black cat to a car a other to the other and the other to a day of a other and the other to the other and the other to the other of the other and a other
------------ temperature: 0.5
-----  Generating with Seed: "on the inside an american"


on the inside an american to see and a new time i a little better to do to get you a other to a own baby are the mouth is a 

In [16]:
!ls

OSError: [Errno 12] Cannot allocate memory

In [49]:
padded_gen_text

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,  83,   6, 530,   2, 190,  50,  50,  50,  50,  50,  50,
         50]], dtype=int32)

In [51]:
preds

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [52]:
preds.max()

0.35043529

In [53]:
np.argmax(preds)

50

In [42]:
preds.shape

(70649,)